## RAG-agent

什么时候需要用到RAG-Agent？

当我们在使用RAG的时候我们经常会遇到以下问题：
1. 有时候提问想要提问联网的问题，有时候想要提问知识库的问题，如何判断是联网还是知识库？
2. 提问的问题答案明明在知识库存在，但是提问的方式不对，导致无法回答
3. 检索的内容是正确的，但是模型没有理解对，还是自己编造

当你遇到以下问题的时候，就可以考虑使用搭建各种RAGAgent，从而提高RAG的灵活度和准确度

下面的所有Agent均基于Langraph（一个基于langchain的Agent框架）

### Langraph概念介绍：
1. State（用于存储大模型工作流的中间数据）
2. Node（工作流过程中的处理器，可以是大模型也可以是函数，有输入，有输出）
3. Edge（用来判断不同情款要流向哪个node）
   
了解完这些概念再来看代码就比较简单

### 支持联网搜索的RAGAgent

<img src="/data/xieyu/Teaching/RAG/imgs/web_search_rag.png" alt="向量检索流程图" width="80%" style="display: block; margin: auto;">

In [1]:
## 配置联网工具
from langchain_community.tools.tavily_search import TavilySearchResults
import os
os.environ["TAVILY_API_KEY"] = "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"
web_search_tool = TavilySearchResults(k=3)

In [2]:
## 初始化agent中间存储的数据结构

from typing import List
from typing_extensions import TypedDict
from langchain.retrievers import ParentDocumentRetriever
from langchain.storage import InMemoryStore

class GraphState(TypedDict):
    """
    表示图的状态。

    属性:
        question: 问题
        generation: 大语言模型生成的内容
        documents: 文档列表
    """

    question: str
    generation: str
    documents: List[str]

In [15]:
## 定义一个用来判断问题是否需要联网搜索的处理器
from typing import Literal
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

class RouteQuery(BaseModel):
    """将用户查询路由到最相关的数据源"""
    datasource: Literal["db_search", "web_search"] = Field(
        ...,
        description="根据用户问题选择使用web搜索还是向量数据库"
    )

def route_question(state: dict) -> str:
    """
    根据问题内容决定使用web搜索还是RAG检索
    
    参数:
        state (dict): 当前图状态，包含用户问题
        
    返回:
        str: 下一个调用节点的名称('web_search'或'db_search')
    """
    print("---ROUTE QUESTION---")
    
    # 初始化路由系统
    llm = ChatOpenAI(model="gpt-4o-eus2", temperature=0, base_url="your_base_url", api_key="your_key")
    router = llm.with_structured_output(RouteQuery)
    
    # 设置路由提示模板
    prompt = ChatPromptTemplate.from_messages([
        ("system", """您是一位专门负责将用户问题分流到向量数据库或网络搜索的专家。
        向量数据库中包含RAG的相关知识
        对于这些主题的问题请使用向量数据库，其他问题则使用网络搜索。"""),
        ("human", "{question}")
    ])
    
    # 执行路由判断
    question_router = prompt | router
    result = question_router.invoke({"question": state["question"]})
    
    # 根据结果返回对应路由
    if result.datasource == "web_search":
        print("---正在将问题路由至网络搜索---")
        return "web_search"
    print("---正在将问题路由至向量数据库---")
    return "db_search"

In [16]:
## 定义一个知识库检索的处理节点

from langchain_core.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

def setup_vectorstore():
    """
    初始化并返回向量数据库
    """
    # 配置嵌入模型
    embeding_model = HuggingFaceEmbeddings(
        model_name="/data/lilk/RAG/models/embeding/bge-m3",
        model_kwargs={"device": "cuda"},
        encode_kwargs={"normalize_embeddings": True}
    )

    # 加载向量数据库
    vector_store = FAISS.load_local(
        index_name="rag_teach",
        folder_path="/data/xieyu/Teaching/RAG/data_retriever/rag_teach",
        embeddings=embeding_model,
        allow_dangerous_deserialization=True
    )
    
    return vector_store

# Start of Selection
def db_search(state):
    """
    知识库检索节点，包含检索和大模型回答流程
    
    Args:
        state (dict): 包含用户问题的状态字典
        
    Returns:
        dict: 包含检索结果、原始问题和大模型回答的字典
    """
    print("---知识库检索开始---")
    
    # 获取向量数据库实例
    vector_store = setup_vectorstore()
    
    # 初始化大模型
    llm = ChatOpenAI(
        model_name="gpt-4o-eus2",
        temperature=0.7,
        base_url="your_base_url",
        api_key="your_key"
    )
    
    # 创建回答模板
    rag_template = ChatPromptTemplate.from_messages([
        ("system", """您是一位知识渊博的助手。请基于提供的上下文信息，
                     准确、简洁地回答用户的问题。如果无法从上下文中找到答案，
                     请明确说明。"""),
        ("user", "问题：{question}\n\n上下文：{context}")
    ])
    
    # 设置并行检索流程
    setup_and_retrieval = RunnableParallel(
        {"context": vector_store.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        ),
         "question": RunnablePassthrough()}
    )
    
    # 构建完整的RAG链
    rag_chain = setup_and_retrieval | rag_template | llm
    
    # 执行检索和回答
    result = rag_chain.invoke(state["question"])
    print("---知识库检索和回答完成---")
    
    # 获取原始检索结果
    retrieval_result = setup_and_retrieval.invoke(state["question"])
    
    return {
        "documents": retrieval_result["context"],
        "question": retrieval_result["question"],
        "answer": result.content
    }

In [17]:
## 定义一个联网搜索的处理节点
from langchain_core.documents import Document


def web_search(state):
    """
    联网搜索并使用大模型分析结果的处理节点
    
    Args:
        state (dict): 包含用户问题的状态字典
        
    Returns:
        dict: 包含网络搜索结果、原始问题和大模型回答的字典
    """
    print("---联网搜索开始---")
    question = state["question"]
    
    # 初始化大模型
    llm = ChatOpenAI(
        model_name="gpt-4o-eus2",
        temperature=0.7,
        base_url="your_base_url",
        api_key="your_key"
    )
    
    # 创建回答模板
    web_template = ChatPromptTemplate.from_messages([
        ("system", """您是一位专业的网络信息分析助手。请基于提供的搜索结果，
                     对用户的问题进行全面的分析和回答。如果搜索结果不足以回答问题，
                     请明确指出。"""),
        ("user", "问题：{question}\n\n搜索结果：{context}")
    ])
    
    # 执行网络搜索
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    
    # 构建并执行分析链
    web_chain = web_template | llm
    result = web_chain.invoke({
        "context": web_results.page_content,
        "question": question
    })
    
    print("---联网搜索和分析完成---")
    
    return {
        "documents": web_results,
        "question": question,
        "answer": result.content
    }


In [18]:
## 流程合并
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("web_search", web_search)  # web search
workflow.add_node("db_search", db_search)  # vectorstore

## 配置判断条件：用户输入就到达route_question节点，根据返回值判断是进入web_search还是vectorstore
workflow.add_conditional_edges(
    START,
    route_question,
    {
        "web_search": "web_search",
        "db_search": "db_search",
    },
)

# Compile
app = workflow.compile()

## 呈现流程结果

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

In [8]:
app.invoke({"question": "今天天气怎么样？"})

---ROUTE QUESTION---
---正在将问题路由至网络搜索---
---联网搜索开始---
---联网搜索和分析完成---


{'question': '今天天气怎么样？',
 'documents': Document(page_content="中国 - 今天的详细天气预报。 所有地点的综合天气报告。\n0° -1~8° 0~10° 1~13° 0~14° 顺义 0~10° 怀柔 -1~9° 通州 -1~9° 昌平 -2~9° 延庆 -4~8° 大兴 -1~10° 房山 -1~10° 密云 -1~10° 门头沟 0~9° 平谷 -4~10° 朝阳 0~9° 丰台 0~10° 石景山 0~10° 海淀 -1~10° 东城 0~9° 西城 -1~10° 北京天气 上海天气 西安天气 沈阳天气 天津天气 重庆天气 海宁天气 本溪天气 海城天气 辽源天气 惠州天气 宜昌天气 张家港天气 营口天气 太原天气 长治天气 深圳天气 潍坊天气 葫芦岛天气 海安天气 定州天气 运城天气 襄阳天气 漯河天气 南昌天气 伊春天气 昆山天气 平度天气 宜兴天气 聊城天气 济南天气 枣阳天气 扬州天气 七台河天气 淮北天气 大理天气 北京天气 上海天气 香港天气 新加坡天气 东京天气 首尔天气 孟买天气 吉隆坡天气 雅加达天气 曼谷天气 台北天气 马尼拉天气 新德里天气 迪拜天气 利雅得天气 班加罗尔天气 科威特天气 吉达天气 广州天气 贝鲁特天气 卡拉奇天气 胡志明市天气 钦奈天气 卡萨布兰卡天气 麦纳麦天气 深圳天气 加尔各答天气 安曼天气 伊斯兰堡天气 多哈天气 阿拉木图天气 全国天气 国际天气 历史天气 景点天气 万年历 天气资讯 空气质量 天气地图 手机网页版 手机APP官网 天气插件 XML地图 更正您的地理位置 意见反馈 客服电话：4000002345（工作日：09：00-18：00） 客服邮箱：kefu@2345.com\n纽约, 纽约, 美国 - 当前温度和天气状况。今天详细的每小时天气预报 - 包括天气状况、温度、压力、湿度、降水、露点、风、能见度和紫外线指数数据。\n北 4\xa0公里/小时 北 4\xa0公里/小时 北 5\xa0公里/小时 北 4\xa0公里/小时 北 5\xa0公里/小时 北 5\xa0公里/小时 北 6\xa0公里/小时 北 6\xa0公里/小时 北 6\xa0公里/小时 西北偏西 5\xa0公里/小时 大风西北偏西 5\xa0公里/小时 大风西南偏南 7\xa0公

In [20]:
app.invoke({"question": "RAG是什么？"})

---ROUTE QUESTION---
---正在将问题路由至向量数据库---
---知识库检索开始---


/data/lilk/RAG/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


---知识库检索和回答完成---


{'question': 'RAG是什么？',
 'documents': [Document(metadata={'source': '/data/xieyu/Teaching/RAG/RAG的道与术.txt'}, page_content='RAG的道与术\n目录\n\n目标读者\n想要开始了解并学习 RAG技术的人\n想要利用RAG实现智能知识问答的人\nRAG是什么？\n定义\nRAG是一种能让大量知识 “活” 起来，人与知识无障碍对话的技术\n背景\nAI大模型已经能够和用户流畅交流，解读知识，总结文章，但由于训练他们的知识是互联网过往的公开内容，当我们有非公开的知识想让AI帮忙解读，总结时就涉及到他们的知识盲区，所以RAG技术就是让他们能够了解到我们的非公开知识，更加精准地回答我们的问题\n场景\n企业知识库\n客服机器人\n产品自动推荐\nRAG主流方案\n核心步骤\n收集数据\n导入数据\n问题检索\n结果评估\n迭代优化\n用dify进行演示\n主流应用\nlangchain\n上手方便，装包随时可以使用\ndbgpt\n比较通用，api使用方便，原生支持graphRAG\nragflow\n文档处理最方便\nQanything\n检索到的重排序效果最好\nfastgpt\n接入模块易操作\ndify\n结合工作流更方便\n制作对应的表格\nRAG实践问题\n核心目标\n提高AI回复知识库相关问题的准确性\n实践痛点\n提问的问题稍微跟原文换一种说法就回答不上来\n根据自己的理解问问题\n根据自己的记忆关键词问问题\n查询到的内容都不是自己要的内容\n图片表格的知识信息无法查询\n对长文本提问总结性问题总是回答不全\n对于查询不到的问题进行编造强行回答\n知识库特别大，难以完全测试来保证准确度\n优化步骤\n知道放什么知识？\n知道怎么放知识？\n知道怎么查知识？\n知道查的对不对？\n实践拆解\n知道放什么知识？\n范围\n尽量只放入跟后续提问相关的内容，无效的内容越多，查询准确率越低\n质量\n内容要求要准确严谨，互相矛盾，避免大模型产生混乱\n类型\n本质上大模型只能够回答文本的内容，所以导入的文档应该尽量转化成普通文本模式\n知识到怎么放知识\n知识类型\n纯文本导入\n带图片文本导入\n带表格文本导入\n导入数据库\nrelation-d

## 拓展Agent


如果发现提问的方式不对导致检索不到可以参考下面这种自动修正RAG-AGENT

1. 能够自动检查检索的知识库内容是否正确
2. 能够判断最终回复的结果是否正确回答问题
3. 能够结果联网进行检索


 ![问题重写](/data/xieyu/Teaching/RAG/imgs/问题重写.png)

代码地址：[extend_adaptive_rag.ipynb](/data/xieyu/Teaching/RAG/rag_agent/extend_adaptive_rag.ipynb)